In [21]:
import keras.backend as K
from keras.models import Model
from keras.layers import Dense, Activation, Flatten, Dropout, Conv2D
from keras.layers import AveragePooling2D, concatenate, Input, Lambda, Cropping2D, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D
from keras.layers import concatenate, Reshape
from keras import utils
from keras.callbacks import Callback, LambdaCallback, EarlyStopping, ModelCheckpoint, TensorBoard, LearningRateScheduler, ReduceLROnPlateau
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from keras.layers.normalization import BatchNormalization
# from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.applications.vgg16 import VGG16, preprocess_input
import numpy as np
import pandas as pd
import cv2
import csv
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import glob
from shutil import copyfile

In [15]:
img_extension = ['.png', '.jpg']
cars = []
notcars = []
for ext in img_extension:
    cars.extend(glob.glob('data/vehicles/**/*' + ext, recursive=True))
    notcars.extend(glob.glob('data/non-vehicles/**/*' + ext, recursive=True))
cars = shuffle(cars)
notcars= shuffle(notcars)

In [18]:
import os, sys
current_dir = os.getcwd()
DATA_HOME_DIR = current_dir+'/data_keras'

In [36]:
# for i in range(int(0.8 * len(cars))): 
#     filename = os.path.basename(os.path.normpath(cars[i]))
#     copyfile(cars[i], DATA_HOME_DIR+'/train/cars/' + filename)

# for i in range(int(0.8 * len(notcars))): 
#     filename = os.path.basename(os.path.normpath(notcars[i]))
#     copyfile(notcars[i], DATA_HOME_DIR+'/train/notcars/' + filename)

In [41]:

train_len_cars = int(0.8 * len(cars))
train_len_notcars = int(0.8 * len(notcars))
# for i in range(int(0.2 * len(cars))): 
#     filename = os.path.basename(os.path.normpath(cars[i + train_len_cars - 1]))
#     copyfile(cars[i + train_len_cars], DATA_HOME_DIR+'/test/cars/' + filename)

# for i in range(int(0.2 * len(notcars))): 
#     filename = os.path.basename(os.path.normpath(notcars[i + train_len_notcars -1]))
#     copyfile(notcars[i  +train_len_notcars], DATA_HOME_DIR+'/test/notcars/' + filename)

In [32]:
cars_test = [slice(int(0.8 * len(cars)), len(cars))]

In [33]:
cars_test[0]

slice(7033, 8792, None)

In [14]:
K.clear_session()
base_model = VGG16(weights='imagenet')
x = base_model.output
# x = Flatten()(x)
# # let's add a fully-connected layer
# x = Dense(256, activation='relu')(x)
# x = Dropout(0.25)(x)
# x = Dense(124, activation='relu')(x)
# x = Dropout(0.25)(x)
# predictions = Dense(1, activation='sigmoid')(x)

# # this is the model we will train
# model = Model(inputs=base_model.input, outputs=predictions)

fc2 = base_model.get_layer('fc2').output
prediction = Dense(output_dim=1, activation='sigmoid', name='logit')(fc2)
model = Model(input=base_model.input, output=prediction)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

/Users/wwymak/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1, activation="sigmoid", name="logit")`
/Users/wwymak/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("lo..., inputs=Tensor("in...)`


In [10]:
img_extension = ['.png', '.jpg']
cars = []
notcars = []
for ext in img_extension:
    cars.extend(glob.glob('data/vehicles/**/*' + ext, recursive=True))
    notcars.extend(glob.glob('data/non-vehicles/**/*' + ext, recursive=True))
cars = shuffle(cars)
notcars= shuffle(notcars)
cars_data = [img_to_array(load_img(img_path, target_size=(64,64))) for img_path in cars]
notcars_data = [img_to_array(load_img(img_path, target_size=(64,64))) for img_path in notcars]
X = np.array(cars_data+notcars_data)
y = np.concatenate((np.ones(len(cars)), np.zeros(len(notcars))))
np.save('traindata_x_small.npy', X)
np.save('traindata_y_small.npy', y)

In [7]:
X = np.load('traindata_x_small.npy')
y = np.load('traindata_y_small.npy')

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [9]:
y_train = utils.to_categorical(y_train, 2)
y_test = utils.to_categorical(y_test, 2)

In [10]:
def preprocess_input_vgg(x):
    """Wrapper around keras.applications.vgg16.preprocess_input()
    to make it compatible for use with keras.preprocessing.image.ImageDataGenerator's
    `preprocessing_function` argument.
    
    Parameters
    ----------
    x : a numpy 3darray (a single image to be preprocessed)
    
    Note we cannot pass keras.applications.vgg16.preprocess_input()
    directly to to keras.preprocessing.image.ImageDataGenerator's
    `preprocessing_function` argument because the former expects a
    4D tensor whereas the latter expects a 3D tensor. Hence the
    existence of this wrapper.
    
    Returns a numpy 3darray (the preprocessed image).
    
    """
    X = np.expand_dims(x, axis=0)
    X = preprocess_input(X)
    return X[0]

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg,
                                   rescale=1. / 255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow(X_train, y_train, batch_size=16)

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg,rescale=1. / 255)
validation_generator = validation_datagen.flow(X_test, y_test)

In [11]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional layers
for layer in model.layers:
    if layer.name in ['fc1', 'fc2', 'logit']:
        continue
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])



In [12]:
# train the model on the new data for a few epochs
model.fit_generator(train_generator,
                    steps_per_epoch=len(X_train) / 16, epochs=1, validation_data=validation_generator,
                    validation_steps=32)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
# for i, layer in enumerate(base_model.layers):
#     print(i, layer.name)

# # we chose to train the top 2 inception blocks, i.e. we will freeze
# # the first 249 layers and unfreeze the rest:
# for layer in model.layers[:249]:
#    layer.trainable = False
# for layer in model.layers[249:]:
#    layer.trainable = True

# # we need to recompile the model for these modifications to take effect
# # we use SGD with a low learning rate
# from keras.optimizers import SGD
# model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# # we train our model again (this time fine-tuning the top 2 inception blocks
# # alongside the top Dense layers
# model.fit_generator(...)


Epoch 1/1
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)
(1, 64, 64, 3)


ValueError: Error when checking input: expected input_3 to have shape (None, 224, 224, 3) but got array with shape (16, 64, 64, 3)

In [26]:
K.clear_session()

In [33]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

In [34]:
a = np.expand_dims(X_train[0], axis=0)
b = preprocess_input(a)
print(b.shape)


(1, 64, 64, 3)


In [36]:
b

array([[[[-49.93900299, -21.77899933, -50.68000031],
         [-48.93900299, -22.77899933, -51.68000031],
         [-45.93900299, -25.77899933, -54.68000031],
         ..., 
         [-39.93900299, -10.77899933, -45.68000031],
         [-40.93900299, -12.77899933, -40.68000031],
         [-40.93900299, -13.77899933, -37.68000031]],

        [[-50.93900299, -21.77899933, -51.68000031],
         [-49.93900299, -22.77899933, -53.68000031],
         [-46.93900299, -25.77899933, -56.68000031],
         ..., 
         [-39.93900299, -10.77899933, -45.68000031],
         [-40.93900299, -12.77899933, -40.68000031],
         [-40.93900299, -13.77899933, -37.68000031]],

        [[-51.93900299, -22.77899933, -52.68000031],
         [-50.93900299, -23.77899933, -54.68000031],
         [-47.93900299, -26.77899933, -57.68000031],
         ..., 
         [-39.93900299, -10.77899933, -45.68000031],
         [-40.93900299, -13.77899933, -41.68000031],
         [-41.93900299, -14.77899933, -40.68000031